In [7]:
import numpy as np # linear algebra
import json
from matplotlib import pyplot as plt
from skimage import color
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score

from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
import get_lodging_scores
import cv2  # importing cv
import imutils
import glob

#We need these in this file:
import load_read_name_extractor as lrne
import os
import pickle

In [8]:
def it_name_extract_labels_from_img(names):
    plots = []
    temp = []
    for i in range(len(names)):
        name_t = os.path.basename(names[i])
        #name_t = name_t.replace(".tif", "")
        for j in range(1,len(name_t)):
            if (names[i][-j] == "_"):
                temp.append(name_t[-(j-1):len(name_t)].replace(".jpeg", ""))
                break
        temp.append(name_t.replace(temp[0], "").replace("_.jpeg", ""))
        plots.append(temp)
        temp = []
    return plots

In [9]:
features_load = np.load('features/hog_features_hog_cells_8x8_block_2x2.npy', allow_pickle=True)
names_load = np.load('features/img_names_hog_cells_8x8_block_2x2.npy', allow_pickle=True)
print(len(features_load[0]))
print(len(names_load))

37332
4983


In [10]:
labels = get_lodging_scores.get_labels()
#[ROI, Visual SCORE, FLIGHTFolder, BIN]
print(features_load[0])
ht_mean = features_load[0].mean(axis=0)
ht_mean

print(labels[0])

[0.25384534 0.13523542 0.0568279  ... 0.08940607 0.16619457 0.06113594]
(61, 50, '160607_Min', 9.0)


In [11]:
names = it_name_extract_labels_from_img(names_load)

In [12]:
def match_pic_label(features, labels, names):
    n = len(labels)
    m = len(names)
    data_frame = []
    for i in range(n):
        for j in range(m):
            if (str(labels[i][0]) == str(names[j][0]) and labels[i][2] == names[j][1]):
                data_frame.append(np.hstack((features[j], labels[i][3])))
    return data_frame

In [13]:
class_weights =    {1:{1:0, 2:1, 3:2, 4:3, 5:4, 6:5, 7:6, 8:7, 9:8, 10:9, 11:10, 12:11, 13:12, 14:13, 15:14, 16:15, 17:16, 18:17, 19:18, 20:19},
                    2:{1:1, 2:0, 3:1, 4:2, 5:3, 6:4, 7:5, 8:6, 9:7, 10:8, 11:9, 12:10, 13:11, 14:12, 15:13, 16:14, 17:15, 18:16, 19:17, 20:18},
                    3:{1:2, 2:1, 3:0, 4:1, 5:2, 6:3, 7:4, 8:5, 9:6, 10:7, 11:8, 12:9, 13:10, 14:11, 15:12, 16:13, 17:14, 18:15, 19:16, 20:17},
                    4:{1:3, 2:2, 3:1, 4:0, 5:1, 6:2, 7:3, 8:4, 9:5, 10:6, 11:7, 12:8, 13:9, 14:10, 15:11, 16:12, 17:13, 18:14, 19:15, 20:16},
                    5:{1:4, 2:3, 3:2, 4:1, 5:0, 6:1, 7:2, 8:3, 9:4, 10:5, 11:6, 12:7, 13:8, 14:9, 15:10, 16:11, 17:12, 18:13, 19:14, 20:15},
                    6:{1:5, 2:4, 3:3, 4:2, 5:1, 6:0, 7:1, 8:2, 9:3, 10:4, 11:5, 12:6, 13:7, 14:8, 15:9, 16:10, 17:11, 18:12, 19:13, 20:14},
                    7:{1:6, 2:5, 3:4, 4:3, 5:2, 6:1, 7:0, 8:1, 9:2, 10:3, 11:4, 12:5, 13:6, 14:7, 15:8, 16:9, 17:10, 18:11, 19:12, 20:13},
                    8:{1:7, 2:6, 3:5, 4:4, 5:3, 6:2, 7:1, 8:0, 9:1, 10:2, 11:3, 12:4, 13:5, 14:6, 15:7, 16:8, 17:9, 18:10, 19:11, 20:12},
                    9:{1:8, 2:7, 3:6, 4:5, 5:4, 6:3, 7:2, 8:1, 9:0, 10:1, 11:2, 12:3, 13:4, 14:5, 15:6, 16:7, 17:8, 18:0, 19:10, 20:11},
                    10:{1:9, 2:8, 3:7, 4:6, 5:5, 6:4, 7:3, 8:2, 9:1, 10:0, 11:1, 12:2, 13:3, 14:4, 15:5, 16:6, 17:7, 18:8, 19:9, 20:10},
                    11:{1:10, 2:9, 3:8, 4:7, 5:6, 6:5, 7:4, 8:3, 9:2, 10:1, 11:0, 12:1, 13:2, 14:3, 15:4, 16:5, 17:6, 18:7, 19:8, 20:9},
                    12:{1:11, 2:10, 3:9, 4:8, 5:7, 6:6, 7:5, 8:4, 9:3, 10:2, 11:1, 12:0, 13:1, 14:2, 15:3, 16:4, 17:5, 18:6, 19:7, 20:8},
                    13:{1:12, 2:11, 3:10, 4:9, 5:8, 6:7, 7:6, 8:5, 9:4, 10:3, 11:2, 12:1, 13:0, 14:1, 15:2, 16:3, 17:4, 18:5, 19:6, 20:7},
                    14:{1:13, 2:12, 3:11, 4:10, 5:9, 6:8, 7:7, 8:6, 9:5, 10:4, 11:3, 12:2, 13:1, 14:0, 15:1, 16:2, 17:3, 18:4, 19:5, 20:6},
                    15:{1:14, 2:13, 3:12, 4:11, 5:10, 6:9, 7:8, 8:7, 9:6, 10:5, 11:4, 12:3, 13:2, 14:1, 15:0, 16:1, 17:2, 18:3, 19:4, 20:5},
                    16:{1:15, 2:14, 3:13, 4:12, 5:11, 6:10, 7:9, 8:8, 9:7, 10:6, 11:5, 12:4, 13:3, 14:2, 15:1, 16:0, 17:1, 18:2, 19:3, 20:4},
                    17:{1:16, 2:15, 3:14, 4:13, 5:12, 6:11, 7:10, 8:9, 9:8, 10:7, 11:6, 12:5, 13:4, 14:3, 15:2, 16:1, 17:0, 18:1, 19:2, 20:3},
                    18:{1:17, 2:16, 3:15, 4:14, 5:13, 6:12, 7:11, 8:10, 9:9, 10:8, 11:7, 12:6, 13:5, 14:4, 15:3, 16:2, 17:1, 18:0, 19:1, 20:2},
                    19:{1:18, 2:17, 3:16, 4:15, 5:14, 6:13, 7:12, 8:11, 9:10, 10:9, 11:8, 12:7, 13:6, 14:5, 15:4, 16:3, 17:2, 18:1, 19:0, 20:1},
                    20:{1:19, 2:18, 3:17, 4:16, 5:15, 6:14, 7:13, 8:12, 9:11, 10:10, 11:9, 12:8, 13:7, 14:6, 15:5, 16:4, 17:3, 18:2, 19:1, 20:0}}

In [14]:
temp = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])

class_weights =    {1:temp}

In [15]:
clf = svm.SVC(kernel="rbf", class_weight="balanced")
features_aar = np.array(features_load)
data_frame = match_pic_label(features_load, labels, names)
print(data_frame[0])
np.random.shuffle(data_frame)
data_frame = np.array(data_frame)


[0.20380611 0.08864094 0.02353812 ... 0.30848572 0.30848572 9.        ]


In [16]:
percentage = 80
partition = int(len(features_aar)*percentage/100)

In [17]:
x_train, x_test = data_frame[:partition,:-1],  data_frame[partition:,:-1]
y_train, y_test = data_frame[:partition,-1:].ravel() , data_frame[partition:,-1:].ravel()
print(y_train)

clf.fit(x_train,y_train)

[13.  7.  0. ...  2.  1.  3.]


SVC(class_weight='balanced')

In [18]:
print(clf)

SVC(class_weight='balanced')


In [19]:
y_pred = clf.predict(x_test)

In [20]:
filename = 'SVM_20bins_cells_8x8_block_2x2_weight_balanced.sav'
pickle.dump(clf, open(filename, 'wb'))

In [21]:
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)

In [22]:
print("Accuracy: "+str(accuracy_score(y_test, y_pred)))
print('\n')
print(classification_report(y_test, y_pred))

Accuracy: 0.4492753623188406


              precision    recall  f1-score   support

         0.0       0.60      0.77      0.67       388
         1.0       0.13      0.25      0.17        60
         2.0       0.06      0.04      0.05        47
         3.0       0.07      0.03      0.04        31
         4.0       0.00      0.00      0.00        23
         5.0       0.28      0.17      0.21        29
         6.0       1.00      0.16      0.28        25
         7.0       0.00      0.00      0.00        32
         8.0       0.00      0.00      0.00        14
         9.0       0.50      0.10      0.16        21
        10.0       0.00      0.00      0.00         8
        11.0       1.00      0.03      0.06        35
        12.0       0.27      0.27      0.27        26
        13.0       0.29      0.30      0.29        33
        14.0       0.27      0.52      0.36        27
        15.0       0.52      0.43      0.47        28
        16.0       0.00      0.00      0.00       

c:\Users\marti\miniconda3\envs\imgp2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\marti\miniconda3\envs\imgp2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\marti\miniconda3\envs\imgp2\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
